In [98]:
import pandas as pd
import numpy as np
import datetime as dt
import quant_risk_mgmt as qrm
from scipy.stats import norm
from scipy.optimize import fsolve, minimize
import inspect

Problem 1 - Calculate greeks with GBSM & Binary Tree
Assume you a call and a put option with the following:

Current Stock Price $151.03

Strike Price $165

Current Date 03/13/2022

Options Expiration Date 04/15/2022

Risk Free Rate of 4.25%

Continuously Compounding Coupon of 0.53%

Implement the closed form greeks for GBSM. Implement a finite difference derivative calculation. Compare the values between the two methods for both a call and a put.

Implement the binomial tree valuation for American options with and without discrete dividends. Assume the stock above:


Pays dividend on 4/11/2022 of $0.88

Calculate the value of the call and the put. Calculate the Greeks of each.

What is the sensitivity of the put and call to a change in the dividend amount?

In [99]:
def black_scholes(S0, K, T, r, q, sigma, option):
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option == 'call':
        price = S0 * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * np.exp(-q * T) * norm.cdf(-d1)
    
    return price

In [100]:
def bt_american_continous(S0, K, T, r, q, sigma, N=200, option_type='call'):
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    pu = (np.exp((r-q)*dt)-d)/(u-d)
    pd = 1-pu
    df = np.exp(-r*dt)
    z = 1 if option_type == 'call' else -1
    def nNodeFunc(n):
        return (n+2)*(n+1)//2
    def idxFunc(i,j):
        return nNodeFunc(j-1)+i
    nNodes = nNodeFunc(N)
    optionValues = np.empty(nNodes, dtype = float)

    for j in range(N, -1, -1):
        for i in range(j, -1, -1):
            idx = idxFunc(i,j)
            price = S0*u**i*d**(j-i)
            optionValues[idx] = max(0,z*(price-K))
            if j < N:
                optionValues[idx] = max(optionValues[idx], df*(pu*optionValues[idxFunc(i+1,j+1)] + pd*optionValues[idxFunc(i,j+1)])  )
    return optionValues[0]

In [101]:
def bt_american_discrete(S0, K, r, T, sigma, N, option_type, dividend_dates=None, dividend_amounts=None):
    if dividend_dates is None or dividend_amounts is None or (len(dividend_amounts)==0) or (len(dividend_dates)==0):
        return bt_american_continous(S0, K, T, r, 0, sigma, N, option_type)
    elif dividend_dates[0] > N:
        return bt_american_continous(S0, K, T, r, 0, sigma, N, option_type)

    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    pu = (np.exp(r*dt)-d)/(u-d)
    pd = 1-pu
    df = np.exp(-r*dt)
    z = 1 if option_type == 'call' else -1
    
    def nNodeFunc(n):
        return (n+2)*(n+1)//2
    def idxFunc(i,j):
        return nNodeFunc(j-1)+i
   
    nDiv = len(dividend_dates)
    nNodes = nNodeFunc(dividend_dates[0])

    optionValues = np.empty(nNodes, dtype = float)

    for j in range(dividend_dates[0],-1,-1):
        for i in range(j,-1,-1):
            idx = idxFunc(i,j)
            price = S0*u**i*d**(j-i)       
            
            if j < dividend_dates[0]:
                #times before the dividend working backward induction
                optionValues[idx] = max(0,z*(price-K))
                optionValues[idx] = max(optionValues[idx], df*(pu*optionValues[idxFunc(i+1,j+1)] + pd*optionValues[idxFunc(i,j+1)])  )
                
            else:
                no_ex= bt_american_discrete(price-dividend_amounts[0], K, r, T-dividend_dates[0]*dt, sigma, N-dividend_dates[0], option_type, [x- dividend_dates[0] for x in dividend_dates[1:nDiv]], dividend_amounts[1:nDiv] )
                ex =  max(0,z*(price-K))
                optionValues[idx] = max(no_ex,ex)

    return optionValues[0]

In [102]:
def first_order_der(func, x, delta):
    return (func(x * (1 + delta)) - func(x * (1 - delta))) / (2 * x * delta)

def second_order_der(func, x, delta):
    return (func(x * (1 + delta)) + func(x * (1 - delta)) - 2 * func(x)) / (x * delta) ** 2

def cal_partial_der(func, order, arg_name, delta=1e-5):
    arg_names = list(inspect.signature(func).parameters.keys())
    derivative_fs = {1: first_order_der, 2: second_order_der}

    def partial_derivative(*args, **kwargs):
        args_dict = dict(list(zip(arg_names, args)) + list(kwargs.items()))
        arg_val = args_dict.pop(arg_name)

        def partial_f(x):
            p_kwargs = {arg_name:x, **args_dict}
            return func(**p_kwargs)
        return derivative_fs[order](partial_f, arg_val, delta)
    return partial_derivative

In [103]:
class option:
    def __init__(self, S, K, T, r, q, sigma, option_type):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.q = q
        self.sigma = sigma
        self.option_type = option_type
        self.d1 = (np.log(self.S / self.K) + (self.r - self.q + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
        self.d2 = self.d1 - self.sigma * np.sqrt(self.T)
    
    def price_euro(self):
        if self.option_type == 'call':
            price = self.S * np.exp(-self.q * self.T) * norm.cdf(self.d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(self.d2)
        elif self.option_type == 'put':
            price = self.K * np.exp(-self.r * self.T) * norm.cdf(-self.d2) - self.S * np.exp(-self.q * self.T) * norm.cdf(-self.d1)
        
        return price
    
    def delta_closed(self):
        if self.option_type == 'call':
            delta = np.exp(-self.q * self.T) * norm.cdf(self.d1)
        elif self.option_type == 'put':
            delta = -np.exp(-self.q * self.T) * norm.cdf(-self.d1)
        
        return delta
    
    def gamma_closed(self):
        gamma = np.exp(-self.q * self.T) * norm.pdf(self.d1) / (self.S * self.sigma * np.sqrt(self.T))
        
        return gamma
    
    def vega_closed(self):
        vega = self.S * np.exp(-self.q * self.T) * norm.pdf(self.d1) * np.sqrt(self.T)
        
        return vega
    
    def theta_closed(self):
        if self.option_type == 'call':
            theta = -self.S * np.exp(-self.q * self.T) * norm.pdf(self.d1) * self.sigma / (2 * np.sqrt(self.T)) - self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(self.d2) + self.q * self.S * np.exp(-self.q * self.T) * norm.cdf(self.d1)
        elif self.option_type == 'put':
            theta = -self.S * np.exp(-self.q * self.T) * norm.pdf(self.d1) * self.sigma / (2 * np.sqrt(self.T)) + self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(-self.d2) - self.q * self.S * np.exp(-self.q * self.T) * norm.cdf(-self.d1)
        
        return theta
    
    def rho_closed(self):
        if self.option_type == 'call':
            rho = self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(self.d2)
        elif self.option_type == 'put':
            rho = -self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-self.d2)

        return rho


    def carry_rho_closed(self):
        if self.option_type == 'call':
            carry_rho =  self.S * self.T * np.exp(-self.q * self.T) * norm.cdf(self.d1)
        elif self.option_type == 'put':
            carry_rho =  - self.S * self.T * np.exp(-self.q * self.T) * norm.cdf(-self.d1)

        return carry_rho

    
    def greeks_closed(self):
        delta = self.delta_closed()
        gamma = self.gamma_closed()
        vega = self.vega_closed()
        theta = self.theta_closed()
        rho = self.rho_closed()
        carry_rho = self.carry_rho_closed()
        
        return {'Delta': delta, 'Gamma': gamma, 'Vega': vega, 'Theta': theta, 'Rho': rho, "Carry Rho": carry_rho}

In [120]:
S0 = 165
K = 165
T = (dt.datetime(2023,4,15) - dt.datetime(2023,3,13)).days / 365
r = 0.0425
q = 0.0053
sigma = 0.2
N = 200
dividend_dates = [round((dt.datetime(2023,4,11)-dt.datetime(2023,3,13)).days/(dt.datetime(2023,4,15)-dt.datetime(2023,3,13)).days*N)]
dividend_amounts = [0.88]

In [125]:
gbsm_call_closed = option(S0, K, T, r, q, sigma, "call")
print(gbsm_call_closed.greeks_closed())

gbsm_put_closed = option(S0, K, T, r, q, sigma, "put")
print(gbsm_put_closed.greeks_closed())

{'Delta': 0.5340091224850149, 'Gamma': 0.040037930803986446, 'Vega': 19.710179716477544, 'Theta': -24.898522316969515, 'Rho': 7.583586080244792, 'Carry Rho': 7.966245676523029}
{'Delta': -0.4655118142202754, 'Gamma': 0.040037930803986446, 'Vega': 19.710179716477544, 'Theta': -18.78699696527723, 'Rho': -7.277010958127815, 'Carry Rho': -6.944415968299725}


In [122]:
american_call_value_without_div = bt_american_discrete(S0, K, r, T, sigma, N, "call")
american_call_value_with_div = bt_american_discrete(S0, K, r, T, sigma, N, "call", dividend_dates, dividend_amounts)
print("Price of call option with no dividend: {:.2f}" .format(american_call_value_without_div))
print("Price of call option with dividend: {:.2f}" .format(american_call_value_with_div))

Price of call option with no dividend: 4.27
Price of call option with dividend: 4.12


In [123]:
american_put_value_without_div = bt_american_discrete(S0, K, r, T, sigma, N, "put")
american_put_value_with_div = bt_american_discrete(S0, K, r, T, sigma, N, "put", dividend_dates, dividend_amounts)
print("Price of put option with no dividend: {:.2f}" .format(american_put_value_without_div))
print("Price of put option with dividend: {:.2f}" .format(american_put_value_with_div))

Price of put option with no dividend: 3.68
Price of put option with dividend: 4.11


In [152]:
class cal_greeks:
    def __init__(self, option_price_func, S, K, r, T, sigma, option_type, q = None, N = None, dividend_dates=None, dividend_amounts=None):
        self.option_price_func = option_price_func
        self.S = S
        self.K = K
        self.r = r
        self.q = q
        self.T = T
        self.sigma = sigma
        self.N = N
        self.option_type = option_type
        self.dividend_dates = dividend_dates
        self.dividend_amounts = dividend_amounts
        
    def __call__(self, *args, **kwargs):
        return self.option_price_func(*args, **kwargs)
    
    def delta(self):
        delta_calculator = cal_partial_der(self.option_price_func, 1, 'S0')
        if self.option_price_func == black_scholes:
            delta = delta_calculator(self.S, self.K, self.r, self.q, self.T, self.sigma, self.option_type)
        elif self.option_price_func == bt_american_continous:
            delta = delta_calculator(self.S, self.K, self.T, self.r, self.q, self.sigma, self.N, self.option_type)
        elif self.option_price_func == bt_american_discrete:
            delta = delta_calculator(self.S, self.K, self.r, self.T, self.sigma, self.N, self.option_type, self.dividend_dates, self.dividend_amounts)
        return delta
    
    def gamma(self):
        gamma_calculator = cal_partial_der(self.option_price_func, 2, 'S0')
        if self.option_price_func == black_scholes:
            gamma = gamma_calculator(self.S, self.K, self.r, self.q, self.T, self.sigma, self.option_type)
        elif self.option_price_func == bt_american_continous:
            gamma = gamma_calculator(self.S, self.K, self.T, self.r, self.q, self.sigma, self.N, self.option_type)
        elif self.option_price_func == bt_american_discrete:
            gamma = gamma_calculator(self.S, self.K, self.r, self.T, self.sigma, self.N, self.option_type, self.dividend_dates, self.dividend_amounts)
        return gamma
    
    def vega(self):
        vega_calculator = cal_partial_der(self.option_price_func, 1, 'sigma')
        if self.option_price_func == black_scholes:
            vega = vega_calculator(self.S, self.K, self.r, self.q, self.T, self.sigma, self.option_type)
        elif self.option_price_func == bt_american_continous:
            vega = vega_calculator(self.S, self.K, self.T, self.r, self.q, self.sigma, self.N, self.option_type)
        elif self.option_price_func == bt_american_discrete:
            vega = vega_calculator(self.S, self.K, self.r, self.T, self.sigma, self.N, self.option_type, self.dividend_dates, self.dividend_amounts)
        return vega
    
    def theta(self):
        theta_calculator = cal_partial_der(self.option_price_func, 1, 'T')
        if self.option_price_func == black_scholes:
            theta = theta_calculator(self.S, self.K, self.r, self.q, self.T, self.sigma, self.option_type)
        elif self.option_price_func == bt_american_continous:
            theta = theta_calculator(self.S, self.K, self.T, self.r, self.q, self.sigma, self.N, self.option_type)
        elif self.option_price_func == bt_american_discrete:
            theta = theta_calculator(self.S, self.K, self.r, self.T, self.sigma, self.N, self.option_type, self.dividend_dates, self.dividend_amounts)
        return -theta
    
    def rho(self):
        rho_calculator = cal_partial_der(self.option_price_func, 1, 'r')
        if self.option_price_func == black_scholes:
            rho = rho_calculator(self.S, self.K, self.r, self.q, self.T, self.sigma, self.option_type)
        elif self.option_price_func == bt_american_continous:
            rho = rho_calculator(self.S, self.K, self.T, self.r, self.q, self.sigma, self.N, self.option_type)
        elif self.option_price_func == bt_american_discrete:
            rho = rho_calculator(self.S, self.K, self.r, self.T, self.sigma, self.N, self.option_type, self.dividend_dates, self.dividend_amounts)
        return rho 
    
    def carry_rho(self):
        carry_rho_calculator = cal_partial_der(self.option_price_func, 1, 'q')
        if self.option_price_func == black_scholes:
            carry_rho = self.rho() - carry_rho_calculator(self.S, self.K, self.r, self.q, self.T, self.sigma, self.option_type)
        elif self.option_price_func == bt_american_continous:
            carry_rho = self.rho() - carry_rho_calculator(self.S, self.K, self.T, self.r, self.q, self.sigma, self.N, self.option_type)
        elif self.option_price_func == bt_american_discrete:
            return
        return carry_rho 
    
    def greeks(self):
        delta = self.delta()
        gamma = self.gamma()
        vega = self.vega()
        theta = self.theta()
        rho = self.rho()
        carry_rho = self.carry_rho() 
        
        return {'Delta': delta, 'Gamma': gamma, 'Vega': vega, 'Theta': theta, 'Rho': rho, "Carry Rho": carry_rho}
        

In [156]:
gbsm_call_closed = option(S0, K, T, r, q, sigma, "call")
gbsm_call_closed_greeks = gbsm_call_closed.greeks_closed()
greeks = pd.DataFrame.from_dict(gbsm_call_closed_greeks, orient='index', columns=['gbsm_closed_call'])

gbsm_put_closed = option(S0, K, T, r, q, sigma, "put")
gbsm_put_closed_greeks = gbsm_put_closed.greeks_closed()
greeks['gbsm_closed_put'] = pd.DataFrame.from_dict(gbsm_put_closed_greeks, orient='index')

gbsm_call_derivative = cal_greeks(black_scholes, S0, K, r, T, sigma, 'call', q)
gbsm_call_derivative_greeks = gbsm_call_derivative.greeks()
greeks['gbsm_derivative_call'] = pd.DataFrame.from_dict(gbsm_call_derivative_greeks, orient='index')

gbsm_put_derivative = cal_greeks(black_scholes, S0, K, r, T, sigma, 'put', q)
gbsm_put_derivative_greeks = gbsm_put_derivative.greeks()
greeks['gbsm_derivative_put'] = pd.DataFrame.from_dict(gbsm_put_derivative_greeks, orient='index')

bt_american_call = cal_greeks(bt_american_discrete, S0, K, r, T, sigma, 'call', 0, N, dividend_dates, dividend_amounts)
bt_american_call_greeks = bt_american_call.greeks()
greeks['bt_american_call'] = pd.DataFrame.from_dict(bt_american_call_greeks, orient='index')

bt_american_put = cal_greeks(bt_american_discrete, S0, K, r, T, sigma, 'put', 0, N, dividend_dates, dividend_amounts)
bt_american_put_greeks = bt_american_put.greeks()
greeks['bt_american_put'] = pd.DataFrame.from_dict(bt_american_put_greeks, orient='index')

greeks

,gbsm_closed_call,gbsm_closed_put,gbsm_derivative_call,gbsm_derivative_put,bt_american_call,bt_american_put
Delta,0.706362,-0.292927,0.756421,-0.237890,7.033031e-01,-0.300620
Gamma,0.024801,0.024801,0.039565,0.039565,3.115041e-09,0.000000
Vega,19.023726,19.023726,9.607762,9.607762,1.923876e+01,19.330693
Theta,-21.315890,-15.988340,-13.548843,-32.940425,-2.185380e+01,-16.151113
Rho,13.055622,-6.299386,4.586537,-1.574575,1.233504e+01,-5.949192
Carry Rho,14.321671,-5.939169,9.441832,-3.101537,NaN,NaN


Problem 2 - Calculate the Value of the Options for AAPL
Using the options portfolios from Problem3 last week (named problem2.csv in this week’s repo) and assuming :


American Options

Current Date 03/03/2023

Current AAPL price is 165

Risk Free Rate of 4.25%

Dividend Payment of $1.00 on 3/15/2023

Using DailyPrices.csv. Fit a Normal distribution to AAPL returns – assume 0 mean return. Simulate AAPL returns 10 days ahead and apply those returns to the current AAPL price (above).

Calculate Mean, VaR and ES.

Calculate VaR and ES using Delta-Normal.

Present all VaR and ES values a $ loss, not percentages. Compare these results to last week’s results.

In [137]:
current_date = dt.datetime(2023,3,3)
S0 = 151.03
r = 0.0425
days_ahead = 0

In [138]:
aapl_option = pd.read_csv('Project/problem2.csv')
aapl_option['ExpirationDate'] = pd.to_datetime(aapl_option['ExpirationDate'])
price = pd.read_csv('Project/DailyPrices.csv')
aapl_return = pd.DataFrame(qrm.return_calculate(price, 'log')["AAPL"])
aapl_norm = aapl_return - aapl_return.mean()

/Users/wangyuanzhe/Desktop/Y1S2/Fintech545/Week07/quant_risk_mgmt.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars[i]] = p2[:,i]


In [139]:
def implied_volatility_bt(S0, K, r, T, price, N, option, dividend_dates=None, dividend_amounts=None):
    f1 = lambda z: (bt_american_discrete(S0, K, r, T, z, N, option, dividend_dates, dividend_amounts)-price)
    return fsolve(f1, x0 = 0.2)[0]

In [140]:
def calculate_portfolio_values(portfolios, underlying_value, days_ahead=0):
    portfolio_values = pd.DataFrame(index=portfolios["Portfolio"].unique(), columns=[underlying_value])
    portfolio_values = portfolio_values.fillna(0)
    
    for i, portfolio in portfolios.iterrows():

        if portfolio["Type"] == "Stock":
            asset_value = underlying_value
            
        else:
            K = portfolio["Strike"]
            T = ((portfolio["ExpirationDate"] - current_date).days - days_ahead) / 365
            price = portfolio["CurrentPrice"]
            dividend_dates = [round(((dt.datetime(2023,3,15) - current_date).days - days_ahead) / ((portfolio["ExpirationDate"] - current_date).days - days_ahead) * N)]
            dividend_amounts = [1]
            sigma = portfolio["ImpliedVol"]
            
            asset_values = []
            for underlying_prices in np.atleast_1d(underlying_value):
                option_values = (bt_american_discrete(underlying_prices, K, r, T, sigma, 50, portfolio.loc['OptionType'].lower(), dividend_dates, dividend_amounts))
                asset_values.append(option_values)
            asset_value = np.array(asset_values)
        
        portfolio_values.loc[portfolio["Portfolio"], :] += portfolio["Holding"] * asset_value
        
    return portfolio_values

In [141]:
def simulate_prices(daily_returns, current_price, days=1, n_simulation = 1000):

    mu, std = norm.fit(daily_returns)
    simulated_returns = np.random.normal(mu, std, (days, n_simulation))
    simulate_prices = current_price * np.exp(simulated_returns.cumsum(axis=0))
    
    return simulate_prices

In [142]:
iv = []

for i, portfolio in aapl_option.iterrows():

    if portfolio["Type"] == "Stock":
        iv.append(None)

    else:
        K = portfolio["Strike"]
        T = ((portfolio["ExpirationDate"] - current_date).days - days_ahead) / 365
        price = portfolio["CurrentPrice"]
        dividend_dates = [round(((dt.datetime(2023,3,15) - current_date).days - days_ahead) / ((portfolio["ExpirationDate"] - current_date).days - days_ahead) * N)]
        dividend_amounts = [1]
        sigma = implied_volatility_bt(S0, K, r, T,  price, 50, portfolio.loc['OptionType'].lower(), dividend_dates, dividend_amounts)
        iv.append(sigma)

aapl_option["ImpliedVol"] = iv

current_values = calculate_portfolio_values(aapl_option, S0, 0)

In [144]:
np.random.seed(0)
underlying_prices = pd.DataFrame(simulate_prices(aapl_norm, S0, 10))
simulate_portfolio_values = calculate_portfolio_values(aapl_option, underlying_prices.loc[9:].values[0], 10)

In [145]:
merged_df = pd.merge(simulate_portfolio_values, current_values, left_index=True, right_index=True)
price_change = merged_df.sub(merged_df[S0], axis=0).drop(S0, axis=1)
price_change.columns = price_change.columns.str[0]

portfolio_metrics = pd.DataFrame(index=aapl_option["Portfolio"].unique(), columns=["Mean", "VaR", "ES"])
portfolio_metrics = portfolio_metrics.fillna(0)

for index, row in price_change.iterrows():
    mean = row.values.mean()
    var = - np.quantile(row.values, 0.05)
    es = -np.mean(row.values[row.values <= -var])
    
    portfolio_metrics.loc[index, "Mean"] = mean
    portfolio_metrics.loc[index, "VaR"] = var
    portfolio_metrics.loc[index, "ES"] = es
    
portfolio_metrics

,Mean,VaR,ES
Straddle,1.734536,1.235151,1.255981
SynLong,-0.799951,18.566214,22.550447
CallSpread,-0.206661,3.934776,4.221094
PutSpread,0.553890,2.683594,2.870239
Stock,-0.260490,17.318129,21.044675
Call,0.467292,6.099346,6.411479
Put,1.267244,4.436432,4.675090
CoveredCall,-0.902307,13.465899,17.089842
ProtectedPut,0.741073,7.578518,7.880233


Delta-Normal

In [146]:
delta_calculator =  cal_partial_der(bt_american_discrete, 1, 'S0')

for i in range(len(aapl_option)):
    if aapl_option.loc[i,"Type"] != "Stock":

        current_date = dt.datetime(2023,3,3)
        N=50
        K = aapl_option.loc[i,"Strike"]
        ExpirationDate = aapl_option.loc[i, "ExpirationDate"]
        T = ((ExpirationDate  - current_date).days ) / 365
        sigma = aapl_option.loc[i,"ImpliedVol"]
        option_type = aapl_option.loc[i,"OptionType"]
        dividend_dates = [round(((dt.datetime(2023,3,15) - current_date).days ) / ((ExpirationDate  - current_date).days ) * N)]
        dividend_amounts = [1]

        aapl_option.loc[i, "Delta"] = delta_calculator(S0, K, r, T, sigma, N, option_type.lower(), dividend_dates, dividend_amounts) * aapl_option.loc[i,'Holding']
    else:
        aapl_option.loc[i, "Delta"] = 1 * aapl_option.loc[i,'Holding']

In [147]:
delta = aapl_option.groupby("Portfolio")['Delta'].sum().apply(lambda x: -x * (underlying_prices.loc[9:].values[0]-151.03))
delta_df = pd.DataFrame(np.array([delta.values[i].reshape(-1) for i in range(len(delta))]), index=delta.keys(), columns=underlying_prices.loc[9:].values[0])

price_change_delta = price_change.add(delta_df)

portfolio_metrics_1 = pd.DataFrame(index=aapl_option["Portfolio"].unique(), columns=["Mean", "VaR", "ES"])
portfolio_metrics_1 = portfolio_metrics_1.fillna(0)

for index, row in price_change_delta.iterrows():
    mean = row.values.mean()
    var = - np.quantile(row.values, 0.05)
    es = -np.mean(row.values[row.values <= -var])
    
    portfolio_metrics_1.loc[index, "Mean"] = mean
    portfolio_metrics_1.loc[index, "VaR"] = var
    portfolio_metrics_1.loc[index, "ES"] = es
    
portfolio_metrics_1

,Mean,VaR,ES
Straddle,1.761717,1.225568,1.249341
SynLong,-0.533543,1.174581,1.415279
CallSpread,-0.131500,0.936325,2.090806
PutSpread,0.488775,0.067056,0.088895
Stock,0.000000,-0.000000,-0.000000
Call,0.614087,0.754647,0.766513
Put,1.147630,0.469972,0.483347
CoveredCall,-0.739787,4.610202,7.438171
ProtectedPut,0.923255,0.478314,0.479471


Problem 3 - Fama, French
Use the Fama French 3 factor return time series (F-F_Research_Data_Factors_daily.CSV) as well as the Carhart Momentum time series (F-F_Momentum_Factor_daily.CSV) to fit a 4 factor model to the following stocks.

AAPL FB UNH MA MSFT
NVDA HD PFE AMZN BRK-B
PG XOM TSLA JPM V
DIS GOOGL JNJ BAC CSCO

Fama stores values as percentages, you will need to divide by 100 (or multiply the stock returns by 100) to get like units.

Based on the past 10 years of factor returns, find the expected annual return of each stock. Construct an annual covariance matrix for the 10 stocks.

Assume the risk free rate is 0.0425. Find the super efficient portfolio

In [130]:
ff = pd.read_csv('Project/F-F_Research_Data_Factors_daily.csv', parse_dates=['Date']).set_index('Date')
mom = pd.read_csv('Project/F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date').rename(columns={'Mom   ':  "Mom"})

factor = (ff.join(mom, how='right') / 100).loc['2013-1-31':]
prices = pd.read_csv('Project/DailyPrices.csv', parse_dates=['Date'])
all_returns = pd.DataFrame(qrm.return_calculate(prices)).set_index('Date')
stocks = ['AAPL', 'META', 'UNH', 'MA',  
          'MSFT' ,'NVDA', 'HD', 'PFE',  
          'AMZN' ,'BRK-B', 'PG', 'XOM',  
          'TSLA' ,'JPM' ,'V', 'DIS',  
          'GOOGL', 'JNJ', 'BAC', 'CSCO']
factors = ['Mkt-RF', 'SMB', 'HML', 'Mom']
dataset = all_returns[stocks].join(factor)

subset = dataset.dropna()

/Users/wangyuanzhe/Desktop/Y1S2/Fintech545/Week07/quant_risk_mgmt.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars[i]] = p2[:,i]


In [131]:
import statsmodels.api as sm
X = subset[factors]
X = sm.add_constant(X)

y = subset[stocks].sub(subset['RF'], axis=0)

betas = pd.DataFrame(index=stocks, columns=factors)
alphas = pd.DataFrame(index=stocks, columns=['Alpha'])


for stock in stocks:
    model = sm.OLS(y[stock], X).fit()
    betas.loc[stock] = model.params[factors]
    alphas.loc[stock] = model.params['const']

sub_return = pd.DataFrame(np.dot(factor[factors],betas.T), index=factor.index, columns=betas.index)
merge_return = pd.merge(sub_return,factor['RF'], left_index=True, right_index=True)
daily_expected_returns = merge_return.add(merge_return['RF'],axis=0).drop('RF',axis=1).add(alphas.T.loc['Alpha'], axis=1)

expected_annual_return = ((daily_expected_returns+1).cumprod().tail(1) ** (1/daily_expected_returns.shape[0]) - 1) * 252

expected_annual_return

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Date,,,,,,,,,,,,,,,,,,,,
2023-01-31,0.157144,0.017941,0.2538,0.222901,0.155944,0.279721,0.120591,0.076962,-0.042945,0.129923,0.08154,0.521821,-0.033253,0.098273,0.241054,-0.155372,-0.017075,0.124206,-0.112301,0.147807


In [132]:
covariance_matrix = dataset[stocks].cov() * 252
covariance_matrix

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
AAPL,0.126877,0.139557,0.037447,0.081272,0.102937,0.171265,0.066193,0.032745,0.122117,0.055520,0.036945,0.037700,0.154880,0.058646,0.071406,0.087399,0.111906,0.022740,0.066245,0.066123
META,0.139557,0.400843,0.017102,0.102465,0.142255,0.240599,0.098845,0.045091,0.194794,0.061619,0.033637,0.020759,0.173121,0.073973,0.085022,0.127047,0.182074,0.021329,0.088791,0.076719
UNH,0.037447,0.017102,0.060922,0.031117,0.036318,0.046531,0.026045,0.032068,0.034737,0.028173,0.027861,0.026843,0.039128,0.033321,0.029590,0.022467,0.029806,0.022981,0.034744,0.028847
MA,0.081272,0.102465,0.031117,0.095762,0.079856,0.137369,0.056792,0.033440,0.096194,0.047520,0.031163,0.030837,0.097521,0.058309,0.082400,0.076987,0.079016,0.017382,0.063399,0.051734
MSFT,0.102937,0.142255,0.036318,0.079856,0.127839,0.175956,0.070916,0.035082,0.133856,0.052676,0.033859,0.031304,0.131911,0.056714,0.068230,0.088227,0.120259,0.020323,0.065226,0.060804
NVDA,0.171265,0.240599,0.046531,0.137369,0.175956,0.403814,0.112055,0.046362,0.221364,0.084290,0.041944,0.054455,0.291392,0.098399,0.118145,0.157837,0.188012,0.021758,0.113397,0.098663
HD,0.066193,0.098845,0.026045,0.056792,0.070916,0.112055,0.097074,0.033271,0.096833,0.042310,0.034414,0.015936,0.077754,0.043555,0.050164,0.063936,0.069626,0.022364,0.046487,0.048412
PFE,0.032745,0.045091,0.032068,0.033440,0.035082,0.046362,0.033271,0.070517,0.037274,0.031202,0.027470,0.019979,0.022283,0.031862,0.030650,0.024522,0.029355,0.027603,0.030812,0.029531
AMZN,0.122117,0.194794,0.034737,0.096194,0.133856,0.221364,0.096833,0.037274,0.242775,0.065772,0.030153,0.037446,0.187717,0.070819,0.083373,0.124253,0.149924,0.022616,0.085241,0.071419
BRK-B,0.055520,0.061619,0.028173,0.047520,0.052676,0.084290,0.042310,0.031202,0.065772,0.050175,0.024839,0.034026,0.061541,0.047076,0.042080,0.051383,0.056281,0.019263,0.050820,0.040421


In [133]:
def super_efficient_portfolio(returns, rf_rate, cov_matrix):
    num_assets = returns.shape[0] if len(returns.shape) == 1 else returns.shape[1]
    
    def neg_sharpe_ratio(weights):
        port_return = np.sum(returns * weights)
        port_std_dev = np.sqrt(weights @ cov_matrix @ weights.T)
        sharpe_ratio = (port_return - rf_rate) / port_std_dev
        return -sharpe_ratio
    
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
                   {'type': 'ineq', 'fun': lambda w: w}]
    
    bounds = [(0, 1) for _ in range(num_assets)]
    
    # Solve for optimal weights
    init_weights = np.ones(num_assets) / num_assets
    opt_result = minimize(neg_sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    
    opt_weights = opt_result.x * 100
    opt_port_return = np.sum(returns * opt_weights)
    opt_port_std_dev = np.sqrt(opt_weights @ cov_matrix @ opt_weights.T)
    opt_sharpe_ratio = (opt_port_return - rf_rate) / opt_port_std_dev
    return opt_weights, opt_sharpe_ratio


In [136]:
weights, sharpe_ratio = super_efficient_portfolio(expected_annual_return.values[0], 0.0425, covariance_matrix)

print("The Portfolio's Sharpe Ratio is: {:.2f}" .format(sharpe_ratio))

weights = pd.DataFrame(weights, index=expected_annual_return.columns, columns=['weight %']).round(2).T
weights

The Portfolio's Sharpe Ratio is: 1.65


,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
weight %,0.0,0.0,22.57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.44,0.0,0.0,12.93,0.0,0.0,7.05,0.0,0.0
